***

*Course:* [Math 535](https://people.math.wisc.edu/~roch/mmids/) - Mathematical Methods in Data Science (MMiDS)  
*Chapter:* 1-Introduction   
*Author:* [Sebastien Roch](https://people.math.wisc.edu/~roch/), Department of Mathematics, University of Wisconsin-Madison  
*Updated:* Feb 11, 2024   
*Copyright:* &copy; 2024 Sebastien Roch

***

In [ ]:
# You will need the files:
#     * mmids.py
#     * penguins-measurements.csv
#     * penguins-species.csv
# from https://github.com/MMiDS-textbook/MMiDS-textbook.github.io/tree/main/utils
#
# IF RUNNING ON GOOGLE COLAB (RECOMMENDED):
# "Upload to session storage" from the Files tab on the left
# Alternative instructions: https://colab.research.google.com/notebooks/io.ipynb

In [ ]:
# PYTHON 3
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
import pandas as pd
import mmids
seed = 535
rng = np.random.default_rng(seed)
import warnings
warnings.filterwarnings('ignore')

## Motivating example: identifying penguin species

Imagine that you are an evolutionary biologist studying penguins. You have collected measurements on a large number of individual specimens. Your goal is to identify different [species](https://en.wikipedia.org/wiki/Species) within this collection based on those measurements.

**Figure:** An Adelie penguin ([Source](https://commons.wikimedia.org/wiki/File:Hope_Bay-2016-Trinity_Peninsula%E2%80%93Ad%C3%A9lie_penguin_%28Pygoscelis_adeliae%29_04.jpg))

![An Adelie penguin](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e3/Hope_Bay-2016-Trinity_Peninsula–Adélie_penguin_%28Pygoscelis_adeliae%29_04.jpg/346px-Hope_Bay-2016-Trinity_Peninsula–Adélie_penguin_%28Pygoscelis_adeliae%29_04.jpg)

$\bowtie$

Here is a [penguin dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set) collected and made available by [Dr. Kristen Gorman](https://www.uaf.edu/cfos/people/faculty/detail/kristen-gorman.php) and the [Palmer Station, Antarctica LTER](https://pallter.marine.rutgers.edu/). We will upload the data in the form of a data table (similar to a spreadsheet) called [`DataFrame`](https://pandas.pydata.org/docs/reference/frame.html) in [`pandas`](https://pandas.pydata.org/docs/), where the columns are different measurements (or features) and the rows are different samples. Below, we load the data using [`pandas.read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html?highlight=read_csv#) and show the first $5$ lines of the dataset (see [`DataFrame.head`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html)). This dataset is a simplified version (i.e., with some columns removed) of the full dataset, maintained by [Allison Horst](https://allisonhorst.com/) at this [GitHub page](https://github.com/allisonhorst/palmerpenguins/blob/main/README.md). 

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('penguins-measurements.csv')
df.head()

Observe that this dataset has missing values (i.e., the entries `NaN` above). A common way to deal with this issue is to remove all rows with missing values. This can be done using [`pandas.DataFrame.dropna`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html). This kind of pre-processing is fundamental in data science, but we will not discuss it much in this course.

In [ ]:
df = df.dropna()
df.head()

There are $342$ samples, as can be seen by using [`pandas.DataFrame.shape`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shape.html) which gives the dimensions of the DataFrame as a tuple.

In [ ]:
df.shape

Here is a summary of the data (see [`pandas.DataFrame.describe`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)).

In [ ]:
df.describe()

Let's first extract the columns into a Numpy array using [`pandas.DataFrame.to_numpy()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.values.html).

In [ ]:
X = df.to_numpy()
print(X)

We visualize two measurements in the data, the bill depth and flipper length. (The original dataset used the more precise term [culmen](https://en.wikipedia.org/wiki/Beak#Culmen) depth.) Below, each point is a sample. This is called a [scatter plot](https://en.wikipedia.org/wiki/Scatter_plot). We use [`matplotlib.pyplot`](https://matplotlib.org/stable/api/pyplot_summary.html) for most of our plotting needs in this book, with a few exceptions (see below). Specifically, here we use the function [`matplotlib.pyplot.scatter`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html).

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(X[:,1], X[:,2], s=10)
plt.xlabel('bill_depth_mm')
plt.ylabel('flipper_length_mm')
plt.show()

We observe what appears to be two fairly well-defined clusters of samples on the top left and bottom right respectively. What is a [cluster](https://en.wikipedia.org/wiki/Cluster_analysis)? Intuitively, it is a group of samples that are close to each other, but far from every other sample. In this case, it may be an indication that these samples come from a separate species.

Now let's look at the full dataset. Visualizing the full $4$-dimensional data is not straightforward. One way to do this is to consider all pairwise scatter plots. We use the function [`seaborn.pairplot`](https://seaborn.pydata.org/generated/seaborn.pairplot.html) from the library [Seaborn](https://seaborn.pydata.org/index.html). 

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(df, height=1.5)
plt.show()

**NUMERIC ANSWER:** How many species of penguins do you think there are in this dataset? $\ddagger$

What would be useful is a method that *automatically* identifies clusters *whatever the dimension of the data*. In this chapter, we will discuss a standard way to do this: $k$-means clustering. We will come back to the penguins dataset later in the chapter. 

But first we need to review some basic concepts about vectors and distances in order to formulate clustering as an appropriate *optimization* problem, a perspective that will be recurring throughout.

**LEARNING BY CHATTING** Ask your favorite AI chatbot for alternative ways to deal with missing values in a dataset. Implement one of these alternatives on the penguins dataset (ask the chatbot for the code). ([Open in Colab](https://colab.research.google.com/github/MMiDS-textbook/MMiDS-textbook.github.io/blob/main/just_the_code/roch_mmids_chap_intro_notebook.ipynb))

$\newcommand{\bmu}{\boldsymbol{\mu}}$ $\newcommand{\bSigma}{\boldsymbol{\Sigma}}$ $\newcommand{\E}{\mathbb{E}}$ $\newcommand{\bX}{\mathbf{X}}$

## Background: quick refresher of matrix algebra, differential calculus, and elementary probability

**NUMERICAL CORNER:** In Numpy, a vector is defined as a 1d array. We first must import the [Numpy](https://numpy.org) package, which is often abbreviated by `np`.

In [ ]:
import numpy as np

In [ ]:
u = np.array([1., 3., 5. ,7.])
print(u)

To obtain the norm of a vector, we can use the function [`linalg.norm`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html), which requires the [`numpy.linalg`](https://numpy.org/doc/stable/reference/routines.linalg.html) package:

In [ ]:
from numpy import linalg as LA

In [ ]:
LA.norm(u)

which we check next "by hand"

In [ ]:
np.sqrt(np.sum(u ** 2))

In Numpy, [`**`](https://numpy.org/doc/stable/reference/generated/numpy.power.html) indicates element-wise exponentiation.

**TRY IT!** Compute the inner product of $u = (1,2,3,4)$ and $v = (5, 4, 3, 2)$ without using the function [`np.dot`](https://numpy.org/doc/stable/reference/generated/numpy.dot.html). *Hint*: The product of two real numbers $a$ and $b$ is `a * b`. ([Open in Colab](https://colab.research.google.com/github/MMiDS-textbook/MMiDS-textbook.github.io/blob/main/just_the_code/roch_mmids_chap_intro_notebook.ipynb))

In [ ]:
u = np.array([1., 2., 3. ,4.])
# EDIT THIS LINE: define v
# EDIT THIS LINE: compute the inner product between u and v

$\unlhd$

**NUMERICAL CORNER:** We will often work with collections of $n$ vectors $\mathbf{x}_1, \ldots, \mathbf{x}_n$ in $\mathbb{R}^d$ and it will be convenient to stack them up into a matrix

$$
X =
\begin{bmatrix}
\mathbf{x}_1^T \\
\mathbf{x}_2^T \\
\vdots \\
\mathbf{x}_n^T \\
\end{bmatrix}
=
\begin{bmatrix}
x_{11} & x_{12} & \cdots & x_{1d} \\
x_{21} & x_{22} & \cdots & x_{2d} \\
\vdots & \vdots & \ddots & \vdots \\
x_{n1} & x_{n2} & \cdots & x_{nd} \\
\end{bmatrix}.
$$

To create a matrix out of two vectors, we use the function [`numpy.stack`](https://numpy.org/doc/stable/reference/generated/numpy.stack.html).

In [ ]:
u = np.array([1., 3., 5., 7.])
v = np.array([2., 4., 6., 8.])
X = np.stack((u,v),axis=0)
print(X)

Quoting the documentation:

> The axis parameter specifies the index of the new axis in the dimensions of the result. For example, if axis=0 it will be the first dimension and if axis=-1 it will be the last dimension.

The same scheme still works with more than two vectors.

In [ ]:
u = np.array([1., 3., 5., 7.])
v = np.array([2., 4., 6., 8.])
w = np.array([9., 8., 7., 6.])
X = np.stack((u,v,w))
print(X)

$\unlhd$

**NUMERICAL CORNER:** In Numpy, the Frobenius norm of a matrix can be computed using the function [`numpy.linalg.norm`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html).

In [ ]:
A = np.array([[1., 0.],[0., 1.],[0., 0.]])
print(A)

In [ ]:
LA.norm(A)

$\unlhd$

**NUMERICAL CORNER:** The function $f(x) = x^2$ over $\mathbb{R}$ has a global minimizer at $x^* = 0$. Indeed, we clearly have $f(x) \geq 0$ for all $x$ while $f(0) = 0$. To plot the function, we use the [matplotlib](https://matplotlib.org) package again, and specifically its function [`matplotlib.pyplot.plot`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html). We also use the function [`numpy.linspace`](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html) to create an array of evenly spaced numbers where we evaluate $f$.

In [ ]:
import matplotlib.pyplot as plt
x = np.linspace(-2,2,100)
y = x ** 2
plt.plot(x,y)
plt.show()

The function $f(x) = e^x$ over $\mathbb{R}$ does not have a global minimizer. Indeed, $f(x) > 0$ but no $x$ achieves $0$. And, for any $m > 0$, there is $x$ small enough such that $f(x) < m$. Note that $\mathbb{R}$ is *not* bounded, therefore the *Extreme Value Theorem* does not apply here.

In [ ]:
x = np.linspace(-2,2,100)
y = np.exp(x)

In [ ]:
plt.plot(x,y)
plt.ylim(0,5)
plt.show()

The function $f(x) = (x+1)^2 (x-1)^2$ over $\mathbb{R}$ has two global minimizers at $x^* = -1$ and $x^{**} = 1$. Indeed, $f(x) \geq 0$ and $f(x) = 0$ if and only $x = x^*$ or $x = x^{**}$.

In [ ]:
x = np.linspace(-2,2,100)
y = ((x+1)**2) * ((x-1)**2)

In [ ]:
plt.plot(x,y)
plt.ylim(0,5)
plt.show()

$\unlhd$

**NUMERICAL CORNER:** We can use simulations to confirm the *Weak Law of Large Numbers*. Recall that a uniform random variable over the interval $[a,b]$ has density

$$
f_{X}(x)
= \begin{cases}
\frac{1}{b-a} & x \in [a,b] \\
0 & \text{o.w.}
\end{cases}
$$

We write $X \sim \mathrm{U}[a,b]$. We can obtain a sample from $\mathrm{U}[0,1]$ by using the function
[`numpy.random.Generator.uniform`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.uniform.html). We must first instantiate a random number generator (RNG) with [`numpy.random.default_rng`](https://numpy.org/doc/stable/reference/random/generator.html) in Numpy. We provide a [seed](https://numpy.org/doc/stable/reference/random/bit_generators/index.html#seeding-and-entropy) as an initial state for the RNG. Using the same seed again ensures reproducibility. 

In [ ]:
seed = 535
rng = np.random.default_rng(seed)
rng.uniform()

Now we take $n$ samples from $\mathrm{U}[0,1]$ and compute their sample mean. We repeat $k$ times and display the empirical distribution of the sample means using an [histogram](https://en.wikipedia.org/wiki/Histogram).

In [ ]:
def lln_unif(n, k):
    sample_mean = [np.mean(rng.random(n)) for i in range(k)]
    plt.hist(sample_mean,bins=15)
    plt.xlim(0,1)
    plt.title(f'n={n}')
    plt.show()

We start with $n=10$.

In [ ]:
lln_unif(10, 1000)

Taking $n$ much larger leads to more concentration around the mean.

In [ ]:
lln_unif(100, 1000)

**TRY IT!** Recall that the cumulative distribution function (CDF) of a random variable $X$ is defined as 

$$
F_X(z) = \mathbb{P}[X \leq z], \qquad \forall z \in \mathbb{R}.
$$

a) Let $\mathcal{Z}$ be the interval where $F_X(z) \in (0,1)$ and assume that $F_X$ is strictly increasing on $\mathcal{Z}$. Let $U \sim \mathrm{U}[0,1]$. Show that 

$$
\mathbb{P}[F_X^{-1}(U) \leq z] = F_X(z).
$$

b) Generate a sample from $\mathrm{U}[a,b]$ for arbitrary $a$, $b$ using `numpy.random.Generator.uniform` and the observation in a). This is called the inverse transform sampling method. ([Open in Colab](https://colab.research.google.com/github/MMiDS-textbook/MMiDS-textbook.github.io/blob/main/just_the_code/roch_mmids_chap_intro_notebook.ipynb))

In [ ]:
a = -1
b = 1
X = rng.uniform()
# EDIT THIS LINE: transform X to obtain a random variable Y ~ U[a,b]

$\unlhd$

**Normal distribution** Recall that a standard Normal variable $X$ has PDF

$$
f_X(x) 
= \frac{1}{\sqrt{2 \pi}}
\exp\left(
- x^2/2
\right).
$$

Its mean is $0$ and its variance is $1$. 

**NUMERICAL CORNER:** We plot the PDF of a standard normal distribution. We use the function [scipy.stats.norm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html) from the [SciPy library](https://scipy.org), which outputs the PDF. The following code was adapted from [here](https://commons.wikimedia.org/wiki/File:Standard_Normal_Distribution.svg) with the help of ChatGPT.

In [ ]:
from scipy.stats import norm

In [ ]:
# Plot the normal distribution curve
x = np.linspace(-4, 4, 100)
y = norm.pdf(x)
plt.plot(x, y, color='black')

# Fill areas under the curve for different standard deviations
plt.fill_between(x, y, where=(x > -1) & (x < 1), color='red', alpha=0.25)
plt.fill_between(x, y, where=(x > -2) & (x < 2), color='red', alpha=0.25)
plt.hlines(norm.pdf(1), -1, 1, color='black', linestyle='dashed')
plt.hlines(norm.pdf(2), -2, 2, color='black', linestyle='dashed')
plt.text(0, norm.pdf(1) + 0.01, "68.3%", ha='center')
plt.text(0, norm.pdf(2) + 0.01, "95.4%", ha='center')

# Set labels, title, and xticks
plt.ylabel("PDF")
plt.xticks(range(-4, 5, 2), [f'{i}' for i in range(-4, 5, 2)])
plt.show()

$\unlhd$

**NUMERICAL CORNER:** The following function generates $n$ data points from a spherical $d$-dimensional Gaussians with variance $\sigma^2$ and mean $\bmu$.

Below, [`rng.normal(0,1,d)`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.normal.html) generates a `d`-dimensional spherical Gaussian with mean $\mathbf{0}$.

In [ ]:
def spherical_gaussian(d, n, mu, sig):
    X = np.stack(
        [mu + (sig ** 2) * rng.normal(0,1,d) for _ in range(n)]
    )
    return X

We generate $100$ data points in dimension $d=2$. We take $\sigma^2 = 1$ and $\bmu = w \mathbf{e}_1$. Below we use the function [`numpy.concatenate`](https://numpy.org/doc/stable/reference/generated/numpy.concatenate.html) to create a vector by concatenating two given vectors. We use `[w]` to create a vector with a single entry `w`.  We also use the function [`numpy.zeros`](https://numpy.org/doc/stable/reference/generated/numpy.zeros.html) to create an all-zero vector. 

In [ ]:
d, n, w = 2, 100, 3.
sig = 1
mu = np.concatenate(([w], np.zeros(d-1)))
X = spherical_gaussian(d, n, mu, sig)

In [ ]:
plt.scatter(X[:,0], X[:,1], marker='x')
plt.axis('equal')
plt.show()

$\unlhd$

**NUMERICAL CORNER:** This is straightforward to implement by using [`numpy.random.Generator.choice`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.choice.html) to choose the component of each sample. For convenience, we will stack the means and covariances into one array with a new dimension. So, for instance, the covariance matrices will now be in a 3d-array, that is, an array with $3$ indices. The first index corresponds to the component (here $0$ or $1$). Here is an example (where we use [`numpy.identity`](https://numpy.org/doc/stable/reference/generated/numpy.identity.html) to produce an identity matrix):

In [ ]:
sigma1 = np.identity(d)
sigma2 = 0.5 * np.identity(d)
sigma = np.stack((sigma1,sigma2))

In [ ]:
print(sigma[0,:,:])

In [ ]:
print(sigma[1,:,:])

Here is a visualization borrowed from the [TensorFlow tutorial](https://www.tensorflow.org/guide/tensor#basics).

**Figure:** Three matrices ([Source](https://www.tensorflow.org/guide/tensor#basics))

![Three matrices](https://www.tensorflow.org/static/guide/images/tensor/3-axis_numpy.png)

$\bowtie$

**Figure:** Three matrices stacked into a 3d-array ([Source](https://www.tensorflow.org/guide/tensor#basics))


![Three matrices stacked into a tensor](https://www.tensorflow.org/static/guide/images/tensor/3-axis_front.png)

$\bowtie$

The code is the following. It returns an `d` by `n` array `X`, where each row is a sample from a 2-component Gaussian mixture. 

In [ ]:
def gmm2(d, n, phi1, phi2, mu1, sigma1, mu2, sigma2):
    
    # merge components into tensors
    phi = np.stack((phi1, phi2))
    mu = np.stack((mu1, mu2))
    sigma = np.stack((sigma1,sigma2))
    X = np.zeros((n,d))
    
    # choose components of each data point, then generate samples
    component = rng.choice(2, size=n, p=[phi1, phi2])
    for i in range(n):
        X[i,:] = rng.multivariate_normal(
            mu[component[i],:],
            sigma[component[i],:,:])
    
    return X

Let us try it with following parameters.

In [ ]:
d, n, w = 2, 1000, 3.
phi1 = 0.9
phi2 = 0.1
mu1 = np.concatenate(([w], np.zeros(d-1)))
mu2 = np.concatenate(([-w], np.zeros(d-1)))
sigma1 = np.identity(d)
sigma2 = 0.5 * np.identity(d)

In [ ]:
X = gmm2(d, n, phi1, phi2, mu1, sigma1, mu2, sigma2)

In [ ]:
plt.scatter(X[:,0], X[:,1], marker='x')
plt.axis('equal')
plt.show()

As expected, we observe two clusters. The one on the right (component $0$) is denser (i.e., it contains more data points) since `phi0` is much larger than `phi1`. It is also larger as its variances (i.e., the diagonal entries of its covariance matrix) are larger. 

$\unlhd$

## Clustering: an objective, an algorithm and a guarantee

**NUMERICAL CORNER:** Here's a numerical example. We first define a quadratic function.

In [ ]:
def q(a, b, c, x):
    return a * (x ** 2) + b * x + c

We plot it for different values of the coefficients. 

In [ ]:
x = np.linspace(-2, 2, 100)

plt.plot(x, q(2, 4, -1, x))
plt.plot(x, q(2, -4, 4, x))
plt.plot(x, q(-2, 0, 4, x))

plt.legend(['y1', 'y2', 'y3'])

plt.show()

$\unlhd$

We are now ready to describe the <a href="https://en.wikipedia.org/wiki/K-means_clustering">$k$-means algorithm</a>, also known as Lloyd's algorithm. We start from a random assignment of clusters. (An alternative [initialization strategy](https://en.wikipedia.org/wiki/K-means_clustering#Initialization_methods) is to choose $k$ representatives at random among the data points.) We then alternate between the optimal choices in the lemmas. In lieu of pseudo-code, we write out the algorithm in Python. 

The input `X` is assumed to be a collection of $n$ vectors $\mathbf{x}_1, \ldots, \mathbf{x}_n \in \mathbb{R}^d$ stacked into a matrix, with one row for each data point. The other input, `k`, is the desired number of clusters. There is an optional input `maxiter` for the maximum number of iterations, which is set to $10$ by default.

We first define separate functions for the two main steps. To find the minimum of an array, we use the function [`numpy.argmin`](https://numpy.org/doc/stable/reference/generated/numpy.argmin.html). We also use [`numpy.linalg.norm`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html) to compute the Euclidean distance.

In [ ]:
def opt_reps(X, k, assign):
    (n, d) = X.shape
    reps = np.zeros((k, d))
    for i in range(k):
        in_i = [j for j in range(n) if assign[j] == i]             
        reps[i,:] = np.sum(X[in_i,:],axis=0) / len(in_i)
    return reps

def opt_clust(X, k, reps):
    (n, d) = X.shape
    dist = np.zeros(n)
    assign = np.zeros(n, dtype=int)
    for j in range(n):
        dist_to_i = np.array([LA.norm(X[j,:] - reps[i,:]) for i in range(k)])
        assign[j] = np.argmin(dist_to_i)
        dist[j] = dist_to_i[assign[j]]
    G = np.sum(dist ** 2)
    print(G)
    return assign

The main function follows. Below, `rng.integers(0,k,n)` is an array of `n` uniformly chosen integers between `0` and `k-1` (inclusive). (See [random.Generator.integers](https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.integers.html) for details.)

In [ ]:
def kmeans(X, k, maxiter=10):
    (n, d) = X.shape
    assign = rng.integers(0,k,n)
    reps = np.zeros((k, d), dtype=int)
    for iter in range(maxiter):
        # Step 1: Optimal representatives for fixed clusters
        reps = opt_reps(X, k, assign) 
        # Step 2: Optimal clusters for fixed representatives
        assign = opt_clust(X, k, reps) 
    return assign

**LEARNING BY CHATTING** AI chatbots can serve as great personal tutors. For instance, they can provide additional information about the code in this book. Just copy-paste a piece of code and ask "What is this Python code doing?" Don't hesitate to ask follow-up questions. [Here is an example using ChatGPT 4](https://chat.openai.com/share/66f9e704-c8e8-4489-9ab6-8e57740cab3b). 

**NUMERICAL CORNER:** We apply our implementation of $k$-means to the example above. We fix `k` to $3$. Here the data matrix `X` is the following:

In [ ]:
X = np.array([[1., 0.],[-2., 0.],[-2.,1.],[1.,-3.],
              [-10.,10.],[2.,-2.],[-3.,1.],[3.,-1.]])
assign = kmeans(X, 3)

We vizualize the output by coloring the points according to their cluster assignment.

In [ ]:
plt.scatter(X[:,0], X[:,1], c=assign)
plt.show()

We can compute the final representatives (optimal for the final assignment) by using the subroutine `opt_reps`.

In [ ]:
print(opt_reps(X, 3, assign))

Each row is the center of the corresponding cluster. Note these match with the ones we previously computed. Indeed, the clustering is the same (although not necessarily in the same order).

$\unlhd$

We will test our implementation of $k$-means on the penguins dataset introduced earlier in the chapter.

We first extract the columns and combine them into a data matrix `X`. As we did previously, we also remove the rows with missing values.

In [ ]:
df = pd.read_csv('penguins-measurements.csv')
df = df.dropna()
X = df[['bill_length_mm', 'bill_depth_mm', 
        'flipper_length_mm', 'body_mass_g']].to_numpy()

We  visualize a two-dimensional slice of the data. 

In [ ]:
plt.scatter(X[:,1], X[:,3], s=10)
plt.xlabel('bill_depth_mm')
plt.ylabel('body_mass_g')
plt.show()

Observe that the features have quite different scales (tens versus thousands in the plot above). In such a case, it is common to standardize the data so that each feature has roughly the same scale. That is accomplished by, for each column of `X`, subtracting its empirical mean and dividing by its empirical standard deviation. We do this next. 

In [ ]:
mean = np.mean(X, axis=0)  # Compute mean for each column
std = np.std(X, axis=0)  # Compute standard deviation for each column
X = (X - mean) / std # Standardize each column

Now we run the $k$-means algorithm with $k=2$ clusters. 

In [ ]:
assign = kmeans(X, 2)

We vizualize the output as we did before.

In [ ]:
plt.scatter(X[:,1], X[:,3], c=assign, s=10)
plt.xlabel('bill_depth_mm (standardized)')
plt.ylabel('body_mass_g (standardized)')
plt.show()

This clustering looks quite good. Nevertheless recall that:

1. in this plot we are looking at only two of the four variables while $k$-means uses all of them, 

2. we are not guaranteed to find the best solution, 

3. our objective function is somewhat arbitrary, and 

4. it is not clear what the right choice of $k$ is. 

In fact, the original dataset provided the correct answer. Despite what the figure above may lead us to believe, there are in reality three separate species. So let's try with $k=3$ clusters.

In [ ]:
assign = kmeans(X, 3)

The output does not seem quite right.

In [ ]:
plt.scatter(X[:,1], X[:,3], c=assign, s=10)
plt.xlabel('bill_depth_mm (standardized)')
plt.ylabel('body_mass_g (standardized)')
plt.show()

But, remembering the warnings mentioned previously, let's look at a different two-dimensional slice.

In [ ]:
plt.scatter(X[:,0], X[:,3], c=assign, s=10)
plt.xlabel('bill_length_mm (standardized)')
plt.ylabel('body_mass_g (standardized)')
plt.show()

Let's load up the truth and compare. We only keep those samples that were not removed because of missing values (see [`pandas.DataFrame.iloc`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html)).

In [ ]:
df_truth = pd.read_csv('penguins-species.csv') 
df_truth = df_truth.iloc[df.index]
df_truth.head()

The species are:

In [ ]:
species = df_truth['species']
print(species.unique())

To plot the outcome, we color the species blue-green-red using a [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries).

In [ ]:
species2color = {'Adelie': 'b', 'Chinstrap': 'g', 'Gentoo': 'r'}
truth = species.replace(species2color)

Finally, we can compare the output to the truth. The match is quite excellent -- but not perfect.

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharex=True, sharey=True)
ax1.scatter(X[:,0], X[:,3], c=truth, s=3)
ax1.set_title('truth')
ax2.scatter(X[:,0], X[:,3], c=assign, s=3)
ax2.set_title('kmeans')
plt.show()

It is informative to revisit the all pairwise scatter plots using [`seaborn.pairplot`](https://seaborn.pydata.org/generated/seaborn.pairplot.html), updated with this new information via its option `hue`. We first concatenate the dataframes with [`pandas.concat`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html#pandas.concat).

In [ ]:
df_full = pd.concat([df, df_truth], axis=1)
df_full.head()

In [ ]:
import seaborn as sns
sns.pairplot(df_full, hue="species", diag_kind="hist", height=1.5)
plt.show()

Determining the appropriate number of clusters is not a straighforward problem. To quote [Wikipedia](https://en.wikipedia.org/wiki/Determining_the_number_of_clusters_in_a_data_set):

> The correct choice of $k$ is often ambiguous, with interpretations depending on the shape and scale of the distribution of points in a data set and the desired clustering resolution of the user. In addition, increasing $k$ without penalty will always reduce the amount of error in the resulting clustering, to the extreme case of zero error if each data point is considered its own cluster (i.e., when $k$ equals the number of data points, $n$). Intuitively then, the optimal choice of $k$ will strike a balance between maximum compression of the data using a single cluster, and maximum accuracy by assigning each data point to its own cluster. If an appropriate value of $k$ is not apparent from prior knowledge of the properties of the data set, it must be chosen somehow. There are several categories of methods for making this decision.


In practice, [several heuristics](https://en.wikipedia.org/wiki/Determining_the_number_of_clusters_in_a_data_set) are in use. Other approaches to clustering, e.g. [DBSCAN](https://en.wikipedia.org/wiki/DBSCAN) and [hierarchical clustering](https://en.wikipedia.org/wiki/Hierarchical_clustering), do not require a number of clusters as input. 

**TRY IT!** Run the analysis again, but this time *without the standardization step*. What do you observe? Is one feature more influential on the final output than the others? Why do you think that is? ([Open in Colab](https://colab.research.google.com/github/MMiDS-textbook/MMiDS-textbook.github.io/blob/main/just_the_code/roch_mmids_chap_intro_notebook.ipynb))

**TRY IT!** Modify `kmeans` to take a tolerance `tol` as input and stop when the improvement in objective value `G` falls below the tolerance. ([Open in Colab](https://colab.research.google.com/github/MMiDS-textbook/MMiDS-textbook.github.io/blob/main/just_the_code/roch_mmids_chap_intro_notebook.ipynb))

$\unlhd$

## Some observations about high-dimensional data

In this section, we test our implementation of $k$-means on a simple simulated dataset in high dimension.

The following function generates $n$ data points from a mixture of two equally likely, spherical $d$-dimensional Gaussians with variance $1$, one with mean $-w\mathbf{e}_1$ and one with mean $w \mathbf{e}_1$. We use `gmm2` from a previous section. It is found in `mmids.py`, which can be downloaded [here](https://raw.githubusercontent.com/MMiDS-textbook/MMiDS-textbook.github.io/main/utils/mmids.py).

In [ ]:
def two_mixed_clusters(d, n, w):
    
    # set parameters
    phi1 = 0.5
    phi2 = 0.5
    mu1 = np.concatenate(([w], np.zeros(d-1)))
    mu2 = np.concatenate(([-w], np.zeros(d-1)))
    sigma1 = np.identity(d)
    sigma2 = np.identity(d)
    
    return mmids.gmm2(d, n, phi1, phi2, mu1, sigma1, mu2, sigma2)

**Two dimensions** We start with $d=2$.

In [ ]:
d, n, w = 2, 100, 3.
X = two_mixed_clusters(d, n, w)

We use a scatterplot to visualize the data. Each dot corresponds to one data point. Observe the two clearly delineated clusters.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,aspect='equal')
ax.scatter(X[:,0], X[:,1], s=10)
plt.show()

Let's run $k$-means on this dataset using $k=2$. We use `kmeans()` from the `mmids.py` file.

In [ ]:
assign = mmids.kmeans(X, 2)

Our default of $10$ iterations seem to have been enough for the algorithm to converge. We can visualize the result by [coloring](https://matplotlib.org/stable/api/_as_gen/matplotlib.lines.Line2D.html) the points according to the assignment.  

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,aspect='equal')
ax.scatter(X[:,0], X[:,1], c=assign, s=10)
plt.show()

**General dimension** Let's see what happens in higher dimension. We repeat our experiment with $d=1000$.

In [ ]:
d, n, w = 1000, 100, 3.
X = two_mixed_clusters(d, n, w)

Again, we observe two clearly delineated clusters.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,aspect='equal')
ax.scatter(X[:,0], X[:,1], s=10)
plt.show()

This dataset is in $1000$ dimensions, but we've plotted the data in only the first two dimensions. If we plot in any two dimensions not including the first one instead, we see only one cluster. 

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,aspect='equal')
ax.scatter(X[:,1], X[:,2], s=10)
plt.show()

Let's see how $k$-means fares on this dataset.

In [ ]:
assign = mmids.kmeans(X, 2)

Our attempt at clustering does not appear to have been successful.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,aspect='equal')
ax.scatter(X[:,0], X[:,1], c=assign, s=10)
plt.show()

Our attempt at clustering does not appear to have been successful. What happened? While these clusters are easy to tease apart *if we know to look at the first coordinate only*, in the full space the within-cluster and between-cluster distances become harder to distinguish: the noise overwhelms the signal. 

The function below plots the histograms of within-cluster and between-cluster distances for a sample of size $n$ in $d$ dimensions with a given offset. As $d$ increases, the two distributions become increasingly indistinguishable. Later in the course, we will develop dimension-reduction techniques that help deal with this issue. This time, we need labeled data points, i.e., we need to know the component from which each data point comes from. For this purpose, we implement a variant of our previous mixture simulator. This one generates -- separately -- each Gaussian and concatenates the data. So the first `n` samples are from one component and the next `n` samples are from the other.

In [ ]:
def one_cluster(d, n, w):
    X = np.stack(
        [np.concatenate(([w], np.zeros(d-1))) + rng.normal(0,1,d) for _ in range(n)]
    )
    return X

def two_clusters(d, n, w):
    X1 = one_cluster(d, n, -w)
    X2 = one_cluster(d, n, w)
    return X1, X2

In [ ]:
def highdim_2clusters(d, n, w):
    # generate datasets
    X1, X2 = two_clusters(d, n, w)
    
    # within-cluster distances for X1
    intra = np.stack(
        [LA.norm(X1[i,:] - X1[j,:]) for i in range(n) for j in range(n) if j>i]
    )
    plt.hist(intra, density=True, label='within-cluster')
    plt.title(f'dim={d}')
 
    # between-cluster distances
    inter = np.stack(
        [LA.norm(X1[i,:] - X2[j,:]) for i in range(n) for j in range(n)]
    )
    plt.hist(inter, density=True, alpha=0.75, label='between-cluster')

    plt.legend(loc='upper right')
    plt.title(f'dim={d}')

Next we plot the results for dimensions $d=2, 100, 1000$. What do you observe?

In [ ]:
highdim_2clusters(2, 100, 3)

In [ ]:
highdim_2clusters(100, 100, 3)

In [ ]:
highdim_2clusters(1000, 100, 3)

As the dimension increases, the distributions of intra-cluster and inter-cluster distances overlap significantly and become more or less indistinguishable. That provides some insights into why clustering may fail here. Note that we used the same offset for all simulations. On the other hand, if the separation between the clusters is sufficiently large, one would expect clustering to work even in high dimension. 

**TRY IT!** What precedes (and what follows in the next subsection) is not a formal proof that $k$-means clustering will be unsuccessful here. The behavior of the algorithm is quite complex and depends, in particular, on the initialization and the density of points. Here, increasing the number of data points eventually leads to a much better performance. Explore this behavior on your own by modifying the code. (For some theoretical justifications (beyond this course), see [here](https://arxiv.org/pdf/0912.0086.pdf) and [here](http://www.stat.yale.edu/~pollard/Papers/Pollard81AS.pdf).)

**NUMERICAL CORNER:** We can check the theorem in a simulation. Here we pick $n$ points uniformly at random in the $d$-cube $\mathcal{C}$, for a range of dimensions up to `dmax`. We then plot the frequency of landing in the inscribed $d$-ball $\mathcal{B}$ and see that it rapidly converges to $0$. Alternatively, we could just plot the formula for the volume of $\mathcal{B}$. But knowing how to do simulations is useful in situations where explicit formulas are unavailable or intractable.

In [ ]:
def highdim_cube(dmax, n):
    
    in_ball = np.zeros(dmax)
    for d in range(dmax):
        # recall that d starts at 0 so we add 1 below
        in_ball[d] = np.mean(
            [(LA.norm(rng.random(d+1) - 1/2) < 1/2) for _ in range(n)]
        )
    
    plt.plot(np.arange(1,dmax+1), in_ball) 
    plt.xlabel('dim')
    plt.ylabel('in-ball freq')
    plt.show()

We plot the result up to dimension $10$.

In [ ]:
highdim_cube(10, 1000)

$\unlhd$

**NUMERICAL CORNER:** We check our claim in a simulation. We generate standard Normal $d$-vectors using the `rng.normal(0,1,d)` function and plot the histogram of their $2$-norm.

In [ ]:
def normal_shell(d, n):
    one_sample_norm = [LA.norm(rng.normal(0,1,d)) for _ in range(n)]
    plt.hist(one_sample_norm, bins=20)
    plt.xlim=(0,np.stack(one_sample_norm).max())
    plt.show()

We first plot it in one dimensions.

In [ ]:
normal_shell(1, 10000)

In higher dimension:

In [ ]:
normal_shell(100, 10000)

$\unlhd$